In [ ]:
import os
import pandas as pd
data_dir = '../input/sample_images/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('../input/stage1_labels.csv', index_col=0)
import dicom
import numpy as np
import cv2
import math
IMG_PX_SIZE = 150
HM_SLICES = 20
#patient
#label = labels_df.get_value(patient, 'cancer')
path = data_dir + patients[0]
path
slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
#new_slices = []
slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

chunk_sizes = math.ceil(len(slices) / HM_SLICES)
chunk_sizes

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
s=0

a=chunks(slices, chunk_sizes)
for slice_chunk in chunks(slices, chunk_sizes):
    s=s+len(slice_chunk)
    #print(slice_chunk)
    print(len(slice_chunk))
s    

In [ ]:
new_slices = []
def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
IMG_PX_SIZE = 150
HM_SLICES = 20
chunk_sizes = math.ceil(len(slices) / HM_SLICES)    
chunk_sizes

In [ ]:
new_slices = []
def mean(l):
    return sum(l) / len(l)
def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
IMG_PX_SIZE = 150
HM_SLICES = 20
chunk_sizes = math.ceil(len(slices) / HM_SLICES)    
chunk_sizes
for slice_chunk in chunks(slices, chunk_sizes):
    print (len(slice_chunk))
    print('SC Before:',slice_chunk)
    slice_chunk = list(map(mean, zip(*slice_chunk)))
    print('SC After:',len(slice_chunk))
    new_slices.append(slice_chunk)
    print('new slices length:',len(new_slices))
    
#slice_chunk = list(map(mean, zip(*slice_chunk)))
#new_slices.append(slice_chunk)

In [ ]:
type(new_slices[0][0])

In [ ]:
len(new_slices)

In [ ]:
len(patients)

In [ ]:
labels_df = pd.read_csv('../input/stage1_labels.csv', index_col=0)

In [ ]:
for patient in patients[:10]:
    try:
        label = labels_df.get_value(patient, 'cancer')
        path = data_dir + patient
        slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        chunk_sizes = math.ceil(len(slices) / HM_SLICES)


        for slice_chunk in chunks(slices, chunk_sizes):
            slice_chunk = list(map(mean, zip(*slice_chunk)))
            new_slices.append(slice_chunk)

        if len(new_slices) == HM_SLICES-1:
            new_slices.append(new_slices[-1])

        if len(new_slices) == HM_SLICES-2:
            new_slices.append(new_slices[-1])
            new_slices.append(new_slices[-1])

        if len(new_slices) == HM_SLICES+2:
            new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
            del new_slices[HM_SLICES]
            new_slices[HM_SLICES-1] = new_val

        if len(new_slices) == HM_SLICES+1:
            new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
            del new_slices[HM_SLICES]
            new_slices[HM_SLICES-1] = new_val

        print(len(slices), len(new_slices))
    except Exception as e:
        # again, some patients are not labeled, but JIC we still want the error if something
        # else is wrong with our code
        print(str(e))